TODO List:


1) make the checksum ** done xD **

2) change prob to decison(prob)

3) implement Go back N

4) report

5) test with different parameters [still need to fix prob issue first]

6) try sending different files: images,sound files ...etc   [DONE]

7) fix selective repeat at prob <1 xD

8) the hell i see in life :|


#I DOUBT THAT TIMERS THREADS AREN'T ""REASSIGNED""" CASE(PROB<1)
## >>>>>>>>>>>>>>>>>> in cell labeled "HERE" check line 62

#yabni dah selective repeat mesh stop and wait

Download the file to be sent

In [7]:
!wget https://www.dropbox.com/s/g8oq6mm2j1zhkv2/test_file.txt
!wget https://freemusicarchive.org/file/music/Music_for_Video/Podington_Bear/Solo_Instruments/Podington_Bear_-_Starling.mp3

--2018-05-13 16:14:23--  https://www.dropbox.com/s/g8oq6mm2j1zhkv2/test_file.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/rPGnr9QbAWx6UhqHTpNK0rAgsvG1VORkEWq1U5kDSUTjPjszzjMbs1Ah09dr10df/file [following]
--2018-05-13 16:14:23--  https://dl.dropboxusercontent.com/content_link/rPGnr9QbAWx6UhqHTpNK0rAgsvG1VORkEWq1U5kDSUTjPjszzjMbs1Ah09dr10df/file
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.9.6, 2620:100:6031:6::a27d:5106
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.9.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24576 (24K) [text/plain]
Saving to: ‘test_file.txt’

test_file.txt       100%[===================>]  24.00K  --.-KB/s    in 0.02s   

2018-05-13 16:14:24 (1.33 MB

In [0]:
import socket, pickle
receiverIP=socket.gethostbyname(socket.gethostname())
from socket import *
senderPort = 12000
receiverPort = 5200 

In [6]:
!ls

datalab  Podington_Bear_-_Starling.mp3


In [0]:
class Packet:
  def __init__(
  self,
  seq_num=None,
  checksum=None,
  data=None,
  type_ =1,
  act_num=None
  ):
    self.seq_num=seq_num
    self.checksum=checksum
    self.data=data
    self.type_=type_
    self.act_num=act_num
    
  def __lt__(self, other):
        return self.seq_num < other.seq_num

class ACK:
  def __init__(
  self,
  seq_num=None,
  checksum=None,
  real=None
  ):
    self.seq_num=seq_num
    self.checksum=checksum
    self.real=real
  

In [0]:
import random
'''
A function to decide whether to pass the ack/packet or not depending on the probability passed
and a randomly generated number
'''
def decision(probability):
    return random.random() < probability

# Checksum : 

In [22]:
#!pip install hashlib
import struct

f = open("test_file.txt", 'rb')
data = f.read(1)
print(data)
import hashlib
x=hashlib.sha224(data).hexdigest()
y=hashlib.sha224(data).hexdigest()


print(x == y)
'''
def ones_complement(x):
  return x ^ ((1 << x.bit_length()) - 1)

def ones_complement(x):
  return x ^ ((1 << len(x) - 1)

print(ones_complement(data))
'''

'''
def checksum_func(data):
    checksum = 0
    data_len = len(data)
    #if odd number
    if (data_len%2) == 1:
        data_len += 1
        data += struct.pack('!B', 0)
    
    for i in range(0, len(data), 2):
        w = (data[i] << 8) + (data[i + 1])
        checksum += w

    checksum = (checksum >> 16) + (checksum & 0xFFFF)
    checksum = ~checksum&0xFFFF
    return checksum

def compute_checksum(source_port,destination_port,segment_length_bytes,data):
  
'''

b'k'
True


"\ndef checksum_func(data):\n    checksum = 0\n    data_len = len(data)\n    #if odd number\n    if (data_len%2) == 1:\n        data_len += 1\n        data += struct.pack('!B', 0)\n    \n    for i in range(0, len(data), 2):\n        w = (data[i] << 8) + (data[i + 1])\n        checksum += w\n\n    checksum = (checksum >> 16) + (checksum & 0xFFFF)\n    checksum = ~checksum&0xFFFF\n    return checksum\n\ndef compute_checksum(source_port,destination_port,segment_length_bytes,data):\n  \n"

In [0]:
import csv
import 
'''
transform a file to a series of packets
@param sw : boolean whether this is stop and wait or not
@param seq_num_range : maximum value allowed for sequence number to have -1
'''
def file_to_pkt(file_name, sw, seq_num_range):
    packets = []
    f = open(file_name, 'rb')
    data = f.read(1024)
    seqNO = 0
    while data:
        if sw: #make sequence numbers 0 and 1 only in stop and wait
            packets.append(Packet(seq_num=seqNO % 2, checksum=0, data=data,act_num=seqNO))

        else:
            packets.append(Packet(seq_num=seqNO % seq_num_range, checksum=0, data=data,act_num=seqNO))
        data = f.read(1024)
        
        seqNO += 1
    #type 0 is the terminating packet of the file   
    packets[len(packets) - 1].type_ = 0
    return packets
  

def pkt_to_file(file_name, packets):
    f = open(file_name, 'wb')
    for i in range(len(packets)):
        f.write(packets[i].data)
    f.close()
    
def get_stats(file_name,stats_dict):
  with open(file_name, "w") as f:
    writer = csv.writer(f, delimiter="\t", lineterminator="\r\n")
    writer.writerows(stats_dict.items())
    

#HERE

In [0]:
from concurrent.futures import ThreadPoolExecutor, wait, as_completed

from time import sleep
from random import randint
from threading import Thread, Lock

lock=Lock()
'''
ThreadPoolExecutor gives pool of threads specified to the timers
of packets in the window so max no of threads maybe set to window size

concurrent.futures is used to determine whether the function of the thread has completed (returned or not)
function of thread >>> time_alarm(...)

once it is completed (the function of thread) call-back function re-submit the timer if still no ack was received
call-back function >>> resend(...)
>>>> timers[number]=pool.submit(time_alarm, packet, prob, time_out,senderSocket,pool) assigns
the function time_alarm to one thread of the pool with the parameters given

 when the callback function is called >>> timers[item.seq_num].add_done_callback(resend)
once time_alarm finishs its job it passes the returned result to object called "Future" variable: [[fn]] via resend function
resend function reassigns time_alarm to the pool of threads as no ack received and packet is "RESENT"

I THINK THERE IS DEADLOCK HAPPENS IN MIDDLE OF SENDING PROCESS

if u need i can summarize the steps if u want so>>>
give me 20 minutes to scan the code then I will ask you questions
I will understand better this way

'''
def time_alarm(Packet, prob, time_out, senderSocket,pool,timers,dummy):
    time.sleep(time_out)
    print("WOW ",Packet.act_num,dummy)
    return Packet, Packet.seq_num, senderSocket, prob, time_out, pool,timers,dummy

# def resend(senderSocket, timers, prob, pool):
#     for x in as_completed(timers.values()):
#         timers[x.result()[1]]=pool.submit(time_alarm, x.result()[0], prob, time_out)
#         print("GOT--> ",x.result()[0].act_num)
#         if prob:

#             senderSocket.sendto(pickle.dumps(x.result()[0]),(receiverIP, receiverPort))
#             print("resent",x.result()[1],"actual",x.result()[1])
#     print("D.O.N.E")
'''we pass a function to this function? automatically, once time_alarm returns result it is passed 
to callback function resend'''    
def resend(fn):
    
    packet,number,senderSocket, prob, time_out, pool, timers,dummy=fn.result() # fn.result is returned result of time_alarm
    #TODO change prob to decision prob
    if decision(prob):
      with lock:
        try:
          senderSocket.sendto(pickle.dumps(packet),(receiverIP, receiverPort))
          print("resent",number,"actual",packet.act_num)
        except:
          pass
          
          
    else:
    
    timers[number]=pool.submit(time_alarm, packet, prob, time_out,senderSocket,pool,dummy+1) # >>>>>>>>>>>>>>>> TIMERS AREN'T REASSIGNED?????????????????? 

            
            

In [0]:
'''
i'll use those only in case i already received the packet and it is resent to me again
IF THIS PACKET WAS IN THE RANGE OF MY WINDOW OR THAT IN """LAST""" WINDOW SEND ACK AND DON'T ACCEPT IT TWICE
IF PACKET LIES OUTSIDE THIS RANGE IGNORE AT ALL
I understand now xD 
lets go to the meat of the code xD
'''      
def is_in_range(num,start,step_size,range_):
  lim=[ i%range_ for i in range(start,start+step_size)]
  if num in lim:
    return True
  else:
    return False
  
def is_in_lrange(num,start,step_size,range_):
  lim=[ i%range_ for i in range(start-1,start-step_size)]
  if num in lim:
    return True
  else:
    return False
    

          

In [0]:
#@title Parameters Tuning{ form-width: "27%" }
sent_file_name = 'Podington_Bear_-_Starling.mp3'  # @param {type:"string"}
recv_file_name = 'delta.mp3'  # @param {type:"string"}
prob = 1  # @param {type:"slider", min:0, max:1, step:0.01}
window_size = 7  # @param {type:"integer"}
seq_num_range = 14  # @param {type:"integer"}
time_out = 0.01 # @param {type:"number"}

import heapq
import os
import time
#from google.colab import files
import threading
from threading import Thread

#may move this before the function call 
start_time = time.time()

In [0]:
'''I THINK WE FIXED THE PROBLEMS :D'''
''' CAPTAIN KAREEM RERUN THE FILE WITH DIFFERENT PARAMETERS'''
'''ok'''
def recv_child():
    stats_ = {}
    #UDP socket
    receiverSocket = socket(AF_INET, SOCK_DGRAM)
    #reuse address when done with it
    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    receiverSocket.bind(('', receiverPort))

    print ('--The receiver is ready to receive--\n')

    packets = []
    window = []
    received = []
    recv_base = 0
    flag_end = False
#   receiving while loop, message is a packet of the file sent
    while True:
        time.sleep(0.05)
        (message, senderAddress) = receiverSocket.recvfrom(8192)
        #print("GOT SOMETHING")
        message = pickle.loads(message)
        #time instance of receiving the message
        #t6 = time.time()
        #last packet
        if message.type_ == 0:
            flag_end = True
        #if packet received is within current window
        if is_in_lrange(message.seq_num, recv_base, window_size,
                        seq_num_range) or is_in_range(message.seq_num,
                recv_base, window_size, seq_num_range):
            #if packet is not dropped
            if decision(prob):
                #time.sleep(0.004)
                receiverSocket.sendto(pickle.dumps(ACK(message.seq_num,
                        0, message.act_num)), senderAddress)
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver sent ack for packet ' \
                    + str(message.seq_num) + ' actual: ' \
                    + str(message.act_num)
                print (stats_['{0:.6f}'.format(t6 - start_time)])
            #else if packet is dropped
            else:
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - start_time)] = \
                    'receiver dropped ack for packet ' \
                    + str(message.seq_num) + ' actual: ' \
                    + str(message.act_num)

        if message.seq_num in received:
            continue

#         if abs(message.seq_num % seq_num_range - window_size) \
#             in received:
#             received.remove(abs(message.seq_num % seq_num_range
#                             - window_size))

        if message.seq_num<window_size:
          if (message.seq_num+seq_num_range- window_size) in received:
            received.remove(message.seq_num+seq_num_range- window_size)
        else:
          if (message.seq_num-window_size) in received:
            received.remove(message.seq_num-window_size)

        received.append(message.seq_num)

        if recv_base == message.seq_num:
            window.append(message)
            for (i, j) in zip(sorted(window)[0::1], sorted(window)[1::1]):
                if j.seq_num == (i.seq_num + 1) % seq_num_range \
                    or j.seq_num == i.seq_num + 1:
                    recv_base = j.seq_num
                    packets.append(i)
                    window.remove(i)
                   # print (recv_base)
                else:
                    recv_base = (i.seq_num + 1) % seq_num_range
                    packets.append(i)
                    window.remove(i)
                    #print (recv_base)
                    break

            if len(window) == 1 and recv_base == window[0].seq_num:
                recv_base = (window[0].seq_num + 1) % seq_num_range
                packets.append(window[0])
                window.remove(window[0])
                #print (recv_base)

            
        else:
            window.append(message)
            
        if packets:
          if flag_end:
            time.sleep(0.1)

            print ('--Whole file received successfully. File is ready to download--\n')

            get_stats('receiver_stats.txt', stats_)

            pkt_to_file(recv_file_name, packets)
            receiverSocket.close()
            break




In [0]:
#Sender
def send_parent():
    #sender process
    newpid = os.fork()
    #if in child
    if newpid == 0:
        recv_child()
    else:#parent
        stats_ = {}
        time.sleep(1)
        packets = file_to_pkt(file_name=sent_file_name, sw=False,
                              seq_num_range=seq_num_range)

        senderSocket = socket(AF_INET, SOCK_DGRAM)
        senderSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
        senderSocket.bind(('', senderPort))
        

        # ## initial setting of window
        print("length of packets:",len(packets))
        base = 0
        index = 0
        step = 0
        window = []
        timers = {}
        pool = ThreadPoolExecutor(2*window_size)
        not_acked = []
        acked_num = 0
#         t=Thread(target=resend,args=(senderSocket, timers, prob, pool))
#         t.start()
#append and variable change orignal if ERROR
        if index + window_size < len(packets):
            window = packets[index:(index + window_size)]
        else:
            window = packets[index:len(packets)]

        for i in range(len(window)):
            #for each packet in window create an event object and  a timer/thread
            #key of these dics to get these object is packet sequence number
            #senderSocket.sendto(pickle.dumps(window[i]),(receiverIP, receiverPort))
            timers[window[i].seq_num] = pool.submit(time_alarm,window[i],prob,time_out,senderSocket,pool,timers,0)
            timers[window[i].seq_num].add_done_callback(resend)
            not_acked.append(window[i].seq_num)

        while True:

            reply = None
            (reply, receiverAddress) = senderSocket.recvfrom(2048)
            #time.sleep(0.001)
            reply = pickle.loads(reply)
            print("ack seq_num:",reply.seq_num,reply.real,"REC. ACK")
            if isinstance(reply,ACK):
                #time.sleep(0.0001)
                

                if (reply.seq_num in timers.keys()):
                    del timers[reply.seq_num]
                    try:
                      not_acked.remove(reply.seq_num)
                    except ValueError:
                      print(not_acked,reply.seq_num)

                    acked_num += 1
                    print("LENGTH OF PACKETS:",acked_num)

                    if acked_num == len(packets):
                        senderSocket.close()
                        break

                    if base == reply.seq_num:
                        if len(not_acked)==0:
                            step=window_size
                        elif abs(base - heapq.nsmallest(1, not_acked)[0]) > window_size - 1:
                            step = abs(seq_num_range % base - heapq.nsmallest(1, not_acked)[0])
                        elif abs(base - heapq.nsmallest(1, not_acked)[0]) <= window_size - 1:
                            step = abs(base - heapq.nsmallest(1, not_acked)[0])

                        index+=step

                        if index + window_size < len(packets):
                            
                            window = packets[index:index + window_size]
 
                        else:

                            window = packets[index:len(packets)]

                        

                        base= window[0].seq_num
                        print(window[0].act_num," ---- ",window[len(window)-1].act_num)

                        for item in window[-step:]:
                            #senderSocket.sendto(pickle.dumps(item),(receiverIP, receiverPort))
                            timers[item.seq_num]= pool.submit(time_alarm,item,prob,time_out,senderSocket,pool,timers,0)
                            timers[item.seq_num].add_done_callback(resend)
                            not_acked.append(item.seq_num)



        






In [0]:
#function call
send_parent()

In [0]:

!ls

## TO DOWNLOAD THE RECEIVED FILE

In [0]:
from google.colab import files
files.download(recv_file_name)